In [1]:
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
from cicada.communicator import SocketCommunicator

def main(communicator):
    print(f"Hello from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

Hello from player 1!Hello from player 0!



In [3]:
from cicada import transcript
transcript.enable = True

handler = logging.StreamHandler()
handler.setFormatter(transcript.Formatter())
transcript.logger.addHandler(handler)

In [4]:
import numpy

from cicada.additive import AdditiveProtocolSuite

def main(communicator):
    protocol = AdditiveProtocolSuite(communicator)
#    transcript.log(transcript.Category.APP, "Share Secrets", comm=communicator.name, rank=communicator.rank)
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
#    transcript.log(transcript.Category.APP, "Add Secrets", comm=communicator.name, rank=communicator.rank)
    c_share = protocol.add(a_share, b_share)
#    transcript.log(transcript.Category.APP, "Reveal Result", comm=communicator.name, rank=communicator.rank)
    c = protocol.reveal(c_share)
    
SocketCommunicator.run(fn=main, world_size=3);

Player 0 PRZSProtocol __init__
Player 1 PRZSProtocol __init__
Player 2 PRZSProtocol __init__
Player 0 --> 1 PRZS 7343805974754493774
Player 1 --> 2 PRZS 5464314533569539861
Player 2 --> 0 PRZS 7618675081499959052
Player 1 <-- 0 PRZS 7343805974754493774
Player 0 <-- 2 PRZS 7618675081499959052
Player 2 <-- 1 PRZS 5464314533569539861
Player 1 AdditiveProtocolSuite share
Player 0 AdditiveProtocolSuite share
Player 2 AdditiveProtocolSuite share
Player 1 AdditiveProtocolSuite share
Player 2 AdditiveProtocolSuite share
Player 0 Field inplace_add [18446722102890534953, 131072] 18446722102890666025
Player 1 Field inplace_add [10434212987201613578, 196608] 10434212987201810186
Player 2 Field add None None
Player 0 AdditiveProtocolSuite share
Player 1 Field add None None
Player 2 AdditiveProtocolSuite reveal
Player 0 Field add None None
Player 1 AdditiveProtocolSuite reveal
Player 2 --> 0 GATHER 16407681753021415492
Player 0 AdditiveProtocolSuite reveal
Player 1 --> 0 GATHER 4990354836776909419
P

If you want to customize the transcript, you can use a rich set of extra logging fields in your formatting string including: `arrow` (message direction, relative to the local rank), `comm` (communicator name), `dir` (message direction, relative to the local rank), `dst` (message destination), `other` (the other player sending or receiving with the local rank), `payload` (message payload), `rank` (local player), `src` (message source), `tag` (message type), and `verb` ("send" or "receive", depending on whether the local player is sending or receiving).

Because the custom fields used to format the transcript don't exist for other log messages, you'll get exceptions if you try to mix the output from the transcript logger with logging from other modules.  In general, it's a good idea to setup a custom handler for the transcript logger and turn-off propagation, so transcript output can be managed separately from other logging events:

Now, you can change the default formatting to produce a more compact version of the above:

Note in the preceeding log outputs that an entry is generated both when a message is sent *and* when it's received, and that the rank of the  player logging the event is always on the left, whether they are the sender or the receiver.  With the arrows indicating which direction the message is travelling, this is easy for a person to understand, but it makes programmatically parsing the transcript needlessly difficult.  As an alternative, you can adjust the format to always put the sender on the left and the recipient on the right, regardless of which player is logging the event:

Since this format produces two identical events for each message (once when the message is sent, and once when the message is received), you may wish to filter the output to eliminate the duplication, e.g. by only logging messages when they're sent: